In [1]:
# Imports
import keras
from keras.preprocessing.image import ImageDataGenerator
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import os, glob, random, sys, math, cv2
from sklearn.model_selection import train_test_split
from skimage import transform
from tqdm import tqdm

# Model
# from model import Model

%matplotlib inline

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# Read image categories
ROOT_DIR = os.getcwd()
IMAGE_DIR = os.path.join(ROOT_DIR, 'images')
CATEGORIES = [folder for folder in os.listdir(IMAGE_DIR) 
               if os.path.isdir(os.path.join(IMAGE_DIR, folder))]
CATEGORIES.sort() # Alphabetical order
NUM_CLASSES = len(CATEGORIES)
print(CATEGORIES)

['aeschynanthus_parvifolius', 'bougainvillea_glabra', 'costus_woodsonii_maas', 'dillenia_excelsa', 'dillenia_indica', 'hedychium', 'hibiscus_rosa_sinensis', 'ixora_congesta', 'jasminum_sambac', 'lycoris', 'papilionanthe_miss_joaqium', 'plumeria']


In [3]:
IMAGE_DIM = (512, 512,)

data_present = False

if os.path.exists('data'):
    x_train = np.load('data/train_images.npy')
    y_train = np.load('data/train_labels.npy')
    data_present = True
    
else:
    # Read images and labels into arrays
    images = []
    labels = []
    for label, category in enumerate(CATEGORIES):
        print("Processing {} images...".format(category))
        folder_path = os.path.join(IMAGE_DIR, category)
        # Read images in subfolder
        for image_file_path in tqdm(glob.glob(folder_path + "/*.jpg")):
            image_data = cv2.imread(image_file_path)
            if image_data is None:
                print("Faulty image {}; Ignoring...".format(image_file_path))
                continue
            # Resize to constant dimensions
            image_data = transform.resize(image_data, IMAGE_DIM)
            images.append(image_data)
            labels.append(label)

    # Change labels to one-hot vector of length NUM_CLASSES instead of integers
    labels = keras.utils.to_categorical(labels)

    print("Read {} images and {} labels.".format(len(images), len(labels))) # Should be same number

In [4]:
if not data_present:
    # Split into training and test sets
    train_test_split_ratio = 0.9 # Proportion in train set
    x_train, x_test, y_train, y_test = train_test_split(images, labels, train_size = train_test_split_ratio)
    print("{} training images, {} testing images".format(len(x_train), len(x_test)))

    # Convert to numpy arrays
    x_train, x_test, y_train, y_test = np.array(x_train), np.array(x_test), np.array(y_train), np.array(y_test)

In [5]:
if not data_present:
    # Save arrays to file
    if not os.path.exists('data'):
        os.makedirs('data')
    np.save('data/train_images', x_train)
    np.save('data/train_labels', y_train)
    np.save('data/test_images', x_test)
    np.save('data/test_labels', y_test)

In [6]:
# Image data augmentation to increase effective dataset size
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255, # Rescale to range 0..1
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='wrap' # I.e. tiling
)

In [7]:
# Model
base_model = keras.applications.densenet.DenseNet121(include_top=True, weights=None, input_shape=IMAGE_DIM+(3,), classes=NUM_CLASSES)
model = keras.models.Model(inputs=base_model.input, outputs=base_model.get_layer(name='fc1000').output)
model.compile(optimizer = keras.optimizers.Nadam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Callback to save model weights
model_checkpoint = keras.callbacks.ModelCheckpoint('weights.{epoch:02d}.h5', verbose=1)

In [16]:
# Load weights if they exist
model.load_weights('weights.168.h5')

In [ ]:
# Train model

# Hyper parameters
BATCH_SIZE = 8
EPOCHS = 300
STEPS_PER_EPOCH = len(x_train) // BATCH_SIZE

model.fit_generator(
    datagen.flow(x_train, y_train, batch_size = BATCH_SIZE),
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs = EPOCHS,
    callbacks = [model_checkpoint],
    initial_epoch = 168
)

Epoch 169/300
111/111 [==============================] - 166s 1s/step - loss: 0.2542 - acc: 0.9133

Epoch 00169: saving model to weights.169.h5
Epoch 170/300
111/111 [==============================] - 165s 1s/step - loss: 0.2189 - acc: 0.9302

Epoch 00176: saving model to weights.176.h5
Epoch 177/300
111/111 [==============================] - 165s 1s/step - loss: 0.2833 - acc: 0.9020

Epoch 00177: saving model to weights.177.h5
Epoch 178/300
111/111 [==============================] - 165s 1s/step - loss: 0.2204 - acc: 0.9291

Epoch 00178: saving model to weights.178.h5
Epoch 179/300
111/111 [==============================] - 165s 1s/step - loss: 0.2920 - acc: 0.9009

Epoch 00179: saving model to weights.179.h5
Epoch 180/300
111/111 [==============================] - 165s 1s/step - loss: 0.2215 - acc: 0.9257

Epoch 00180: saving model to weights.180.h5
Epoch 181/300
111/111 [==============================] - 165s 1s/step - loss: 0.2832 - acc: 0.9133

Epoch 00181: saving model to weights

Epoch 250/300
111/111 [==============================] - 166s 1s/step - loss: 0.1545 - acc: 0.9403

Epoch 00250: saving model to weights.250.h5
Epoch 251/300
111/111 [==============================] - 165s 1s/step - loss: 0.1773 - acc: 0.9493

Epoch 00251: saving model to weights.251.h5
Epoch 252/300
111/111 [==============================] - 165s 1s/step - loss: 0.1559 - acc: 0.9493

Epoch 00252: saving model to weights.252.h5
Epoch 253/300
111/111 [==============================] - 165s 1s/step - loss: 0.1487 - acc: 0.9527

Epoch 00253: saving model to weights.253.h5
Epoch 254/300
111/111 [==============================] - 165s 1s/step - loss: 0.1275 - acc: 0.9516

Epoch 00254: saving model to weights.254.h5
Epoch 255/300
111/111 [==============================] - 165s 1s/step - loss: 0.1386 - acc: 0.9516

Epoch 00255: saving model to weights.255.h5
Epoch 256/300
 12/111 [==>...........................] - ETA: 2:27 - loss: 0.1202 - acc: 0.9583

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



111/111 [==============================] - 165s 1s/step - loss: 0.1251 - acc: 0.9595

Epoch 00284: saving model to weights.284.h5
Epoch 285/300
111/111 [==============================] - 165s 1s/step - loss: 0.1334 - acc: 0.9550

Epoch 00285: saving model to weights.285.h5
Epoch 286/300
111/111 [==============================] - 165s 1s/step - loss: 0.1852 - acc: 0.9448

Epoch 00286: saving model to weights.286.h5
Epoch 287/300
111/111 [==============================] - 166s 1s/step - loss: 0.1501 - acc: 0.9459

Epoch 00287: saving model to weights.287.h5
Epoch 288/300
111/111 [==============================] - 164s 1s/step - loss: 0.2227 - acc: 0.9257

Epoch 00288: saving model to weights.288.h5
Epoch 289/300
111/111 [==============================] - 166s 1s/step - loss: 0.1272 - acc: 0.9583

Epoch 00289: saving model to weights.289.h5
Epoch 290/300
111/111 [==============================] - 164s 1s/step - loss: 0.0814 - acc: 0.9730

Epoch 00290: saving model to weights.290.h5
Epoch 